### Raw File Download

In [17]:
from bs4 import BeautifulSoup
from datetime import datetime
from pathlib import Path
from lxml import html
import os, requests
import urllib
import pandas as pd
import re

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

C:\Users\31697\AppData\Local\Temp\ipykernel_17020\912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [23]:
dir_path = os.path.dirname(os.getcwd())
data_path = os.path.join(dir_path, 'data')
raw_path = os.path.join(data_path, 'raw')
interim_path = os.path.join(data_path, 'interim')
processed_path = os.path.join(data_path, 'processed')


In [4]:
url = "https://m.rbi.org.in/scripts/WSSViewDetail.aspx?TYPE=Section&PARAM1=7"

In [5]:
main_session = requests.Session()
response = main_session.get(url)
tree = html.fromstring(response.content)

xls_links = [a for a in tree.xpath('//table[@class="tablebg"]//td//a/@href') if a.endswith('XLSX')]
len_links = len(xls_links)
dates = [a for a in tree.xpath('//table[@class="tablebg"]//tr//th/text()')][:len_links]
main_session.close()

# Create a dictionary with dates as keys and links as values
date_link_dict = dict(zip(dates, xls_links))

# Print the resulting dictionary
date_link_dict

{'02 Feb 2024': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_02022024F8FAFE04D30C482C8343900F88301313.XLSX',
 '26 Jan 2024': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_26012024055ABF0D25594708A1C4FB261878E952.XLSX',
 '19 Jan 2024': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_190120247CBC06E61E744F5F85258A067E880D92.XLSX',
 '12 Jan 2024': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_12012024A0B072A073EB466499ED4BE1DB2993F2.XLSX',
 '05 Jan 2024': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_05012024F7945423CC994256B0455B343A2D0EA4.XLSX',
 '29 Dec 2023': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_291220238CFC6B20CE6B45FB84B69A83B8A05047.XLSX',
 '22 Dec 2023': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_22122023312D73530A994A6CA99FD87D3A4603F4.XLSX',
 '15 Dec 2023': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_151220238E0C1A113DBB416EB5C0121CA429F2A2.XLSX',
 '08 Dec 2023': 'https://rbidocs.rbi.org.in/rdocs/Wss/DOCs/6T_08122023BB2BFA22C0C84F8486F20476B833B3AA.XLSX',
 '01 Dec 2

In [8]:
local_files = []
for date, link in date_link_dict.items():
    session = requests.Session()
    resp = session.get(link)
    
    # Format the filename based on the date
    filename_date = date.replace(" ", "_")
    
    local_excel_path = os.path.join(raw_path, f"{filename_date}.xlsx")
    
    with open(local_excel_path, 'wb') as f:
        f.write(resp.content)
    
    session.close()
    local_files.append(local_excel_path)

# Print the list of local file paths
print(local_files)

['d:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\02_Feb_2024.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\26_Jan_2024.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\19_Jan_2024.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\12_Jan_2024.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\05_Jan_2024.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\29_Dec_2023.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stocks\\data\\raw\\22_Dec_2023.xlsx', 'd:\\OneDrive - Indian School of Business\\backup\\idp\\bipp-datasets\\projects\\rbi_money_stock

### Process Raw Data

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
df_list=[]
for file in local_files:
    try:
        print('Processing:',file)
        df = pd.read_excel(file).dropna(how='all', axis=1)
        df=df[3:]
        df=df.fillna(method="ffill",axis=1)
        # Select columns
        filtered = df[[df.columns.tolist()[0], df.columns.tolist()[2]]].copy()
        filtered.columns = ['type','value']
        # Remove rows with NaN
        filtered = filtered.dropna(subset=['value']).reset_index(drop=True)
        # Extract Date of Data
        year=filtered.value.iloc[0]
        date=re.findall("(\w+)",filtered['value'].iloc[1])
        data_date=date[1]+'-'+str(date[0])+'-'+str(year)
        # Remove rows with NaN
        filtered = filtered.dropna(subset=['type']).reset_index(drop=True)
        filtered['type'] = filtered['type'].str.replace(r'[^a-zA-z0-9 ]',' ')
        filtered['date'] = data_date
        # Save as csv file in interim folder
        file_name = file.split('\\')[-1].split('.')[0]
        interim_csv_path = os.path.join(interim_path, f"{file_name}.csv")
        filtered.to_csv(interim_csv_path, index=False)
        print('Processed:',file)
        # # Pivot the table
        # pivoted = filtered.pivot(index='date', columns='type', values='value').copy().reset_index()
        # pivoted.columns = [col.lower().strip().replace(' ','_') for col in pivoted.columns.to_list()]
        df_list.append(filtered)

    except:
        print('Error:',file)

Processing: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\02_Feb_2024.xlsx
Processed: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\02_Feb_2024.xlsx
Processing: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\26_Jan_2024.xlsx
Processed: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\26_Jan_2024.xlsx
Processing: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\19_Jan_2024.xlsx
Processed: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\19_Jan_2024.xlsx
Processing: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stocks\data\raw\12_Jan_2024.xlsx
Processed: d:\OneDrive - Indian School of Business\backup\idp\bipp-datasets\projects\rbi_money_stock

In [80]:
refined_dfs = []

for df in df_list:
    pivoted = df.pivot(index='date', columns='type', values='value').copy().reset_index()
    for col in pivoted.columns:
        if 'Currency with the Public' in col:
            pivoted.rename(columns={col: 'currency_with_public'}, inplace=True)
        elif 'Demand Deposits with Banks' in col:
            pivoted.rename(columns={col: 'demand_deposits_with_banks'}, inplace=True)
        elif 'Time Deposits with Banks' in col:
            pivoted.rename(columns={col: 'time_deposits_with_banks'}, inplace=True)
        elif 'Deposits with Reserve Bank' in col:
            pivoted.rename(columns={col: 'other_deposits_with_rbi'}, inplace=True)
        elif 'M3' in col:
            pivoted.rename(columns={col: 'm3'}, inplace=True)
        
    pivoted = pivoted[['date','currency_with_public','demand_deposits_with_banks','time_deposits_with_banks','other_deposits_with_rbi','m3']]
    refined_dfs.append(pivoted)


pivoted

type,date,currency_with_public,demand_deposits_with_banks,time_deposits_with_banks,other_deposits_with_rbi,m3
0,31-Mar-2017,12637.7,14095.4,101446.8,210.9,128390.8


In [81]:
data = pd.concat(refined_dfs).reset_index(drop=True)
data

type,date,currency_with_public,demand_deposits_with_banks,time_deposits_with_banks,other_deposits_with_rbi,m3
0,12-Jan-2024,3311754,2408079,18411008,78465,24209307
1,12-Jan-2024,3311754,2408079,18411008,78465,24209307
2,29-Dec-2023,3255711,2575881,18336729,81051,24249372
3,29-Dec-2023,3255711,2575881,18336729,81051,24249372
4,15-Dec-2023,3271577,2462242,18159363,75095,23968277
...,...,...,...,...,...,...
331,28-Apr-2017,13539.7,12318.9,100170.6,176.6,126205.9
332,14-Apr-2017,13289.5,12104.5,101221.8,212.8,126828.6
333,14-Apr-2017,13289.5,12104.5,101221.8,212.8,126828.6
334,31-Mar-2017,12637.7,14095.4,101446.8,210.9,128390.8


In [83]:
data = data.drop_duplicates()


In [84]:
data.isna().sum()

type
date                          0
currency_with_public          0
demand_deposits_with_banks    0
time_deposits_with_banks      0
other_deposits_with_rbi       0
m3                            0
dtype: int64

In [85]:
pd.set_option('display.max_rows', None)
data


type,date,currency_with_public,demand_deposits_with_banks,time_deposits_with_banks,other_deposits_with_rbi,m3
0,12-Jan-2024,3311754,2408079,18411008,78465,24209307
2,29-Dec-2023,3255711,2575881,18336729,81051,24249372
4,15-Dec-2023,3271577,2462242,18159363,75095,23968277
6,1-Dec-2023,3250161,2475755,18232186,75584,24033687
8,17-Nov-2023,3253867,2449084,18028414,74364,23805729
10,03-Nov-2023,3206519,2430860,18104162,74276,23815816
12,20-Oct-2023,3201708,2394226,17934407,72940,23603281
14,6-Oct-2023,3188095,2427930,17996638,73449,23686111
16,22-Sep-2023,3195976,2339373,17749388,72741,23357477
18,8-Sep-2023,3209232,2334419,17861719,71801,23477171


In [86]:
pd.reset_option('display.max_rows')

In [89]:
data['date'].value_counts()

date
12-Jan-2024    1
26-Apr-2019    1
25-Oct-2019    1
11-Oct-2019    1
27-Sep-2019    1
              ..
22-Oct-2021    1
8-Oct-2021     1
24-Sep-2021    1
10-Sep-2021    1
31-Mar-2017    1
Name: count, Length: 171, dtype: int64

In [88]:
# drop duplicate rows in date and 1.1_currency_with_the_public is null
data = data.drop_duplicates(subset=['date'])

In [90]:
data

type,date,currency_with_public,demand_deposits_with_banks,time_deposits_with_banks,other_deposits_with_rbi,m3
0,12-Jan-2024,3311754,2408079,18411008,78465,24209307
2,29-Dec-2023,3255711,2575881,18336729,81051,24249372
4,15-Dec-2023,3271577,2462242,18159363,75095,23968277
6,1-Dec-2023,3250161,2475755,18232186,75584,24033687
8,17-Nov-2023,3253867,2449084,18028414,74364,23805729
...,...,...,...,...,...,...
327,26-May-2017,14129.2,12006.1,100916.5,177.7,127229.5
328,12-May-2017,14035.6,12229.5,101583.5,177.5,128026.2
330,28-Apr-2017,13539.7,12318.9,100170.6,176.6,126205.9
332,14-Apr-2017,13289.5,12104.5,101221.8,212.8,126828.6


In [93]:
data.loc[data['date']=='19-July-2019', 'date'] = '19-Jul-2019'

In [96]:
data['date'] = pd.to_datetime(data['date'], format='%d-%b-%Y').dt.strftime('%d-%m-%Y')
data

type,date,currency_with_public,demand_deposits_with_banks,time_deposits_with_banks,other_deposits_with_rbi,m3
0,12-01-2024,3311754,2408079,18411008,78465,24209307
2,29-12-2023,3255711,2575881,18336729,81051,24249372
4,15-12-2023,3271577,2462242,18159363,75095,23968277
6,01-12-2023,3250161,2475755,18232186,75584,24033687
8,17-11-2023,3253867,2449084,18028414,74364,23805729
...,...,...,...,...,...,...
327,26-05-2017,14129.2,12006.1,100916.5,177.7,127229.5
328,12-05-2017,14035.6,12229.5,101583.5,177.5,128026.2
330,28-04-2017,13539.7,12318.9,100170.6,176.6,126205.9
332,14-04-2017,13289.5,12104.5,101221.8,212.8,126828.6


In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 334
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   date                        171 non-null    object
 1   currency_with_public        171 non-null    object
 2   demand_deposits_with_banks  171 non-null    object
 3   time_deposits_with_banks    171 non-null    object
 4   other_deposits_with_rbi     171 non-null    object
 5   m3                          171 non-null    object
dtypes: object(6)
memory usage: 9.4+ KB


In [101]:
num_cols = ['currency_with_public', 'demand_deposits_with_banks', 'time_deposits_with_banks', 'other_deposits_with_rbi', 'm3']
for col in num_cols:
    # Remove commas from column and then convert it to float
    data[col] = data[col].replace(',', '', regex=True).astype(float)
# Check the information of the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 334
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   date                        171 non-null    object 
 1   currency_with_public        171 non-null    float64
 2   demand_deposits_with_banks  171 non-null    float64
 3   time_deposits_with_banks    171 non-null    float64
 4   other_deposits_with_rbi     171 non-null    float64
 5   m3                          171 non-null    float64
dtypes: float64(5), object(1)
memory usage: 9.4+ KB


In [102]:
# Save as csv file in processed folder 
processed_csv_path = os.path.join(processed_path, "rbi_money_stocks.csv")
data.to_csv(processed_csv_path, index=False)